In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import japanize_matplotlib
from IPython.core.display import display

import warnings
warnings.filterwarnings('ignore')

/var/folders/pt/19z35k5s1x76pkdz1df1mnqm0000gn/T/ipykernel_28520/1604713450.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
train = pd.read_csv("csv/train.csv")
# train = pd.read_csv("csv/processed_train.csv")
test = pd.read_csv("csv/test.csv")
# test = pd.read_csv("csv/processed_test.csv")
gender_submission = pd.read_csv("csv/gender_submission.csv")

In [3]:
sample_submission = pd.DataFrame(test["PassengerId"], columns=["PassengerId"])
train = pd.get_dummies(train, columns=["Sex", "Embarked"])
test = pd.get_dummies(test, columns=["Sex", "Embarked"])
train.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis=1, inplace=True)
test.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis=1, inplace=True)

display(train.head())

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [4]:
X_train = train.drop(["Survived"], axis=1)
y_train = train["Survived"]

In [7]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

gbm = lgb.LGBMClassifier(objective="binary")

gbm.fit(train_x, train_y, eval_set=[(train_x,train_y), (valid_x,valid_y)], early_stopping_rounds=20, verbose=10)

oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
print("score", round(accuracy_score(valid_y, oof)*100, 2))

[10]	training's binary_logloss: 0.425134	valid_1's binary_logloss: 0.466511
[20]	training's binary_logloss: 0.345172	valid_1's binary_logloss: 0.414827
[30]	training's binary_logloss: 0.297224	valid_1's binary_logloss: 0.400898
[40]	training's binary_logloss: 0.263737	valid_1's binary_logloss: 0.408284
score 82.37


In [8]:
gbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': 'binary',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [9]:
from sklearn.model_selection import GridSearchCV

gbm = lgb.LGBMClassifier(objective="binary")

params = {
    'num_leaves':[20,31,40,50],
    'reg_alpha':[0,1,10,100],
    'reg_lambda':[0,1,10,100],
}

grid_search = GridSearchCV(
    gbm,
    param_grid=params,
    cv=3,
)
grid_search.fit(X_train,y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

0.830527497194164
{'num_leaves': 20, 'reg_alpha': 0, 'reg_lambda': 10}


In [11]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3, shuffle=True, random_state=0)

score_list = []
test_pred = np.zeros((len(test), 3))

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print(f"fold{fold_+1} start")
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]
    gbm = lgb.LGBMClassifier(objective="binary", num_leaves=20, reg_alpha=0, reg_lambda=10)

    gbm.fit(train_x, train_y, eval_set=[(train_x,train_y), (valid_x,valid_y)], early_stopping_rounds=20, verbose=-1)

    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100, 2))
    test_pred[:,fold_] = gbm.predict_proba(test)[:,1]
    print(f"fold{fold_+1} end\n")
print(score_list, "平均score", np.mean(score_list))

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
sample_submission["Survived"] = pred
sample_submission.to_csv("grid_search.csv", index=False)

fold1 start
fold1 end

fold2 start
fold2 end

fold3 start
fold3 end

[82.15, 83.84, 80.81] 平均score 82.26666666666667
